In [1]:
import re
import random
import numpy as np
import shapefile as spf
from shapely.geometry import shape, Point

In [4]:
def get_rcoordinates(path, country):  
    
    def is_match(text):
        match = re.match(f'.*{country}.*', text)
        return match
    
    # Read the shape file that contains list of countries with their unique ID
    country_list = spf.Reader(path)
    
    # get the record that belong to the country we have queried the function
    country = [c for c in shapes.records() if is_match(c.__str__()) is not None][0] # [0] to extract the only item from the list
    
    # Extract the country ID from the variable country
    country_id = int(re.findall(r'\d+', country.__str__())[0])
    
    # Get the coordinates from the polygon details dictionary
    coordinates_list = country_list.shapeRecords()[country_id].shape.__geo_interface__['coordinates']
    
    coordinates = None
    tol = 100
    count = 0
    while count < tol:
        if len(coordinates_list) != 2:
            coordinates_list = coordinates_list[0]
        else:
            coordinates = coordinates_list   
            break
        count += 1
    
    return coordinates_list

In [143]:
tmp = get_rcoordinates('World_Countries.shp', 'Afghan')

In [144]:
tmp

(61.27655792236328, 35.60724639892578)

In [1]:
!pip install pandas